In [ ]:
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score
from scipy.stats import mode
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import DBSCAN

In [ ]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
import matplotlib.cm as cm
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)

In [ ]:
ss_train_shock = pd.read_csv('ss_train_shock.csv')
ss_test_shock = pd.read_csv('ss_test_shock.csv')
ss_train_shock = ss_train_shock.dropna()
ss_test_shock = ss_test_shock.dropna()
ss_train_shock

In [ ]:
ss_train_shock.describe()

In [ ]:
train = ss_train_shock.drop(ss_train_shock.columns[0], axis=1)
X_train = train.drop(['percent_increase','WEIGHT','HHIDPN','percent_increase','LAST_YEAR', 'SPEND_SS','SHOCK2'], axis=1)
y_train = train['SHOCK2']
other_train = ss_train_shock[['percent_increase','WEIGHT','percent_increase','LAST_YEAR', 'SPEND_SS','SHOCK2','HHIDPN']]

In [ ]:
test = ss_test_shock.drop(ss_test_shock.columns[0], axis=1)
X_test = test.drop(['percent_increase','WEIGHT','HHIDPN','percent_increase','LAST_YEAR', 'SPEND_SS','SHOCK2'], axis=1)
y_test = test['SHOCK2']
other_test = ss_test_shock[['percent_increase','WEIGHT','percent_increase','LAST_YEAR', 'SPEND_SS','SHOCK2','HHIDPN']]

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
other_train

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
other_test

In [ ]:
samples = pd.concat([X_train,X_test])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)

X = X_train_norm
X_test_norm = scaler.transform(X_test)

samples_norm = scaler.transform(samples)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score
from scipy.stats import mode
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

random_state = 42
prp_var = [50,200,300,400,500,600,700]
for i in prp_var:
    tsne = TSNE(n_components=2, init='random', random_state=random_state, perplexity=i,n_jobs=-1)
    X_proj = tsne.fit_transform(samples_norm)
    X_test_tsne = X_proj[-16260:]
    X_train_tsne = X_proj[:37772]
    print(X_test_tsne.shape)
    print(X_train_tsne.shape)
    X_proj = X_train_tsne

    fig = plt.figure(figsize=(12, 6))
    plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], \
            c=[cm.Spectral(float(i)*3 / 8) for i in y_train])
    plt.title("t-SNE transformed data " + str(i))

In [ ]:
prp_var = [800,1000,1200,1400,1600,1800]
for i in prp_var:
    tsne = TSNE(n_components=2, init='random', random_state=random_state, perplexity=i,n_jobs=-1)
    X_proj = tsne.fit_transform(samples_norm)
    X_test_tsne = X_proj[-16260:]
    X_train_tsne = X_proj[:37772]
    print(X_test_tsne.shape)
    print(X_train_tsne.shape)
    X_proj = X_train_tsne

    fig = plt.figure(figsize=(12, 6))
    plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], \
            c=[cm.Spectral(float(i)*3 / 8) for i in y_train])
    plt.title("t-SNE transformed data " + str(i))

In [ ]:
prp_var = [2000,2400,2800]
random_state = 42
for i in prp_var:
    tsne = TSNE(n_components=2, init='random', random_state=random_state, perplexity=i,n_jobs=-1)
    X_proj = tsne.fit_transform(samples_norm)
    X_test_tsne = X_proj[-16260:]
    X_train_tsne = X_proj[:37772]
    print(X_test_tsne.shape)
    print(X_train_tsne.shape)
    X_proj = X_train_tsne

    fig = plt.figure(figsize=(12, 6))
    plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], \
            c=[cm.Spectral(float(i)*3 / 8) for i in y_train])
    plt.title("t-SNE transformed data " + str(i))

In [ ]:
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score
from scipy.stats import mode
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

random_state = 42
prp_var = 1800

# # Transform  dimensional data to 2 dimensions
tsne = TSNE(n_components=2, init='random', random_state=random_state, perplexity=prp_var,n_jobs=-1)
X_proj = tsne.fit_transform(samples_norm)
X_test_tsne = X_proj[-16260:]
X_train_tsne = X_proj[:37772]
print(X_test_tsne.shape)
print(X_train_tsne.shape)
X_proj = X_train_tsne

In [ ]:
fig = plt.figure(figsize=(12, 6))
plt.scatter(X_train_tsne[:, 0], X_train_tsne[:, 1], \
            c=[cm.Spectral(float(i)*3 / 8) for i in y_train])
plt.title("t-SNE Transformed Data, Perplexity = 1800")

In [ ]:
y_train.reset_index(drop=True, inplace=True)

In [ ]:
eps_values = [3, 4, 5, 6, 7, 8]

for eps in eps_values:
    dbs = DBSCAN(eps=eps)
    clusters = dbs.fit_predict(X_proj)

    # Calculate the unique clusters and their counts
    uq = np.unique(clusters)
    num_clusters = len(uq)

    # Permute the labels based on the mode of the original labels
    labels = np.zeros_like(clusters)
    for i in uq:  # Change range(10) to unique values in clusters to handle varying cluster numbers
        mask = (clusters == i)
        # Only assign a label if there are any points in the cluster
        if np.any(mask):
            labels[mask] = mode(y_train[mask])[0][0]

    # Plotting
    fig = plt.figure(figsize=(12, 6))
    plt.scatter(X_proj[:, 0], X_proj[:, 1], c=[cm.Spectral(float(i) / num_clusters) for i in clusters])
    plt.title(f"t-SNE transformed data with EPS = {eps}")
    
    # Calculate and annotate cluster centers
    centers = {}
    for i in uq:
        if i != -1:  # Skip noise points
            center = X_proj[clusters == i].mean(axis=0)
            centers[i] = center
            plt.text(center[0], center[1], str(i), ha="center", va="center", size=15,
                     bbox=dict(boxstyle="circle,pad=0.1", fc="lightblue", ec="steelblue", lw=2))
    
    plt.show()  # Show the plot for each value of eps

    # Print the number of clusters
    print(f"Number of clusters for eps={eps}: {num_clusters}")

    # Compute and print the F1 score
    f1 = f1_score(y_train, labels, average='micro')
    print(f"F1 Score for eps={eps}: {f1}")

In [ ]:
eps = 7

dbs = DBSCAN(eps = eps)

#X_proj = TSNE_DONE[perp][:8000]
clusters = dbs.fit_predict(X_proj)

uq = np.unique(clusters)
num_clusters = len(uq)



#plt.scatter(kmeans.cluster_centers_[:, 0], sc.cluster_centers_[:, 1], s=300, c='black')


# Permute the labels
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(y_train[mask])[0]



fig = plt.figure(figsize=(12, 6))
plt.scatter(X_proj[:, 0], X_proj[:, 1], \
          c=[cm.Spectral(float(i) / num_clusters) for i in clusters])
plt.title("t-SNE Transformed Data for EPS = " + str(eps))
centers = {}
counts = {}
for i in uq:
    count = 0
    center = np.zeros(2)
    for c in range(0, len(clusters)):
        if (clusters[c] != i):
            continue
        count = count + 1
        center = center + X_proj[c]
    if (count > 0):
        counts[i] = count
        centers[i] = center / count

print(num_clusters)

for i in centers.keys():
    if i > 25:
        break
    loc = centers[i]
    plt.text(loc[0], loc[1], str(i), ha="center", va="center", size=15,
            bbox=dict(boxstyle="circle,pad=0.1",
                      fc="lightblue", ec="steelblue", lw=2))
  #plt.show()
# Compute the accuracy
f1_score(y_train, labels, average='micro')

In [ ]:
unique_clusters = np.unique(clusters)


clu_defect = {i: 0 for i in unique_clusters}
clu_total = {i: 0 for i in unique_clusters}


for i, cluster_id in enumerate(clusters):
    if y_train[i]:  
        clu_defect[cluster_id] += 1
    clu_total[cluster_id] += 1

print(clu_total)
print(clu_defect)

# Calculate and print defect percentage for each actual cluster (excluding noise)
for i in unique_clusters:
    if i != -1:  # Skip noise
        total = clu_total[i]
        defect = clu_defect[i]
        defect_percentage = defect / total if total > 0 else 0 
        print(f"{i}th cluster has a total of {total} and a {defect_percentage:.2%} percentage to experience wealth shock")


clu_defect_perc = {i: (clu_defect[i] / clu_total[i] if clu_total[i] > 0 else 0) for i in unique_clusters if i != -1}
for i in sorted(clu_defect_perc.keys(), key=clu_defect_perc.get):
    print(f"{i}: {clu_defect_perc[i]:.2%}")

In [ ]:
print(sum(y_train)/37772)
print(sum(y_test)/16260)

In [ ]:
test_clusters = []

#X_test_tsne = TSNE_DONE[perp][-2000:]

for i in X_test_tsne:
    min = np.linalg.norm(X_train_tsne[0] - i)
    value = 0
    for j in range(0,len(X_train_tsne)):
        if np.linalg.norm(X_train_tsne[j] - i) >= min:
            continue
        min = np.linalg.norm(X_train_tsne[j] - i)
        value = j
    test_clusters.append(clusters[value])
test_clusters = np.array(test_clusters)

plt.scatter(X_test_tsne[:, 0], X_test_tsne[:, 1], \
          c=[cm.Spectral(float(i) / num_clusters) for i in test_clusters])
for i in centers.keys():
    if i > 25:
        break
    loc = centers[i]
    plt.text(loc[0], loc[1], str(i), ha="center", va="center", size=15,
            bbox=dict(boxstyle="circle,pad=0.1",
                      fc="lightblue", ec="steelblue", lw=2))

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
other_train.reset_index(drop=True, inplace=True)
other_test.reset_index(drop=True, inplace=True)

In [ ]:
X_train_clusters = X_train.copy()
X_train_clusters["cluster"] = clusters
Shock_train_clusters = pd.concat([X_train_clusters, y_train, other_train], axis=1)
Shock_train_clusters

In [ ]:
X_test_clusters = X_test.copy()
X_test_clusters["cluster"] = test_clusters
Shock_test_clusters = pd.concat([X_test_clusters, y_test, other_test], axis=1)
Shock_test_clusters

In [ ]:
clu_test_defect = {i:0 for i in range(0,num_clusters)}
clu_test_total = {i:0 for i in range(0,num_clusters)}
for i in range(0,len(test_clusters)):
    if y_test[i]:
        clu_test_defect[test_clusters[i]] += 1
    clu_test_total[test_clusters[i]] += 1

print(clu_test_total)
print(clu_test_defect)

for i in range(num_clusters):
    print("{:d}th cluster is: ".format(i))
    print(X_test.iloc[i])
    print("And has a total of {:d} and a {:f} chance to experience wealth shock".format(clu_test_total[i],clu_test_defect[i]/clu_test_total[i]))
    print("-------------------------------------------------")



clu_test_defect_perc = {i:(clu_test_defect[i]/clu_test_total[i]) for i in range(0,num_clusters)}
for i in (sorted(clu_test_defect_perc.keys(), key=clu_test_defect_perc.get)):
    print("{:d}: {:f}".format(i, clu_test_defect_perc[i]))

print("--------------------")

for i in clu_test_defect_perc.keys():
    print("{:d}: {:f}".format(i, clu_defect_perc[i] - clu_test_defect_perc[i]))

In [ ]:
c_train_data = []

for i in range(num_clusters):
    c_train_data.append(Shock_train_clusters[Shock_train_clusters["cluster"] == i].drop("cluster",axis=1))

for i in range(len(c_train_data)):
    print("CLUSTER {:d}".format(i))
    display(c_train_data[i])
    print("-" * 3600)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

columns = c_train_data[0].columns

for col in columns:
    combined_df = pd.DataFrame()
    
    for i, df in enumerate(c_train_data):
        temp_df = df[[col]].copy()
        temp_df['DataFrame'] = f'Cluster{i+1}'
        combined_df = pd.concat([combined_df, temp_df], ignore_index=True)
    
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='DataFrame', y=col, data=combined_df)
    plt.title(f'Boxplot of {col} across different DataFrames')
    plt.show()

In [ ]:
df_names = ['Low', 'High']  
columns = c_train_data[0].columns
means_df = pd.DataFrame(index=columns, columns=df_names)

for col in columns:
    for name, df in zip(df_names, c_train_data):
        means_df.loc[col, name] = df[col].mean()

print(means_df)

In [ ]:
#High cluster has a total of 785 and a 2.17% percentage to experience wealth shock

#Medium cluster has a total of 27466 and a 1.11% percentage to experience wealth shock

#Low cluster has a total of 6993 and a 0.46% percentage to experience wealth shock

#Age are very similar across all three clsuters
#High cluster experience significantly more drastic health declines and hospitalizations than medium cluster, the same applies to medium and low clusters
#Low cluster has none of the specified diseases
#High cluster has cancer rate of 100%
#Home care, Cancer and HBP are the most significant factors differentiating high, mdeium and low clusters

In [ ]:
c_train_data[0].to_csv('Low_train.csv', index=False)
c_train_data[1].to_csv('High_train.csv', index=False)

In [ ]:
from scipy.spatial import distance_matrix
from skbio import DistanceMatrix
from skbio.stats.distance import permanova
from skbio.stats.distance import anosim

In [ ]:
c_test_data = []

for i in range(num_clusters):
    c_test_data.append(Shock_test_clusters[Shock_test_clusters["cluster"] == i].drop("cluster",axis=1))

for i in range(len(c_test_data)):
    print("CLUSTER {:d}".format(i))
    display(c_test_data[i])
    print("-" * 3600)

In [ ]:
df_names = ['Low', 'High']  
columns = c_test_data[0].columns
means_df = pd.DataFrame(index=columns, columns=df_names)

for col in columns:
    for name, df in zip(df_names, c_test_data):
        means_df.loc[col, name] = df[col].mean()

print(means_df)

In [ ]:
c_test_data[0].to_csv('Low_test.csv', index=False)
c_test_data[1].to_csv('High_test.csv', index=False)

In [ ]:
sample_size = 5000
if len(X_train_norm) > sample_size:
    indices = np.random.choice(len(X_train_norm), size=sample_size, replace=False)
    X_train_sampled = X_train_norm[indices]
else:
    X_train_sampled = X_train_norm

In [ ]:
dm_sampled = DistanceMatrix(distance_matrix(X_train_sampled, X_train_sampled))
clusters_sampled = clusters[indices]
result_anosim = anosim(dm_sampled, clusters_sampled)
print(result_anosim)

In [ ]:
result_permanova = permanova(dm_sampled, clusters_sampled)
print(result_permanova)